In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.7 MB/s eta 0:00:00


In [3]:
from transformers import TFBertForSequenceClassification
import tensorflow as tf

In [4]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import re
from tensorflow import keras
from tqdm import tqdm

In [5]:
from transformers import BertTokenizer

bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [6]:

df_gas = pd.read_csv("/content/drive/MyDrive/NLPicasso/데이터 전처리/가스라이팅 증강.csv")
df_chatbot = pd.read_csv("/content/drive/MyDrive/NLPicasso/데이터 전처리/ChatbotData.csv")
df_gas.columns = ["prompt", "competition",	"label"]
df_chatbot.columns = ["prompt", "competition",	"label"]

In [7]:
df_gas['label'].fillna(1, inplace=True)
df_chatbot['label'] = df_chatbot['label'].replace([1, 2], 0)
df_all = pd.concat([df_gas, df_chatbot], axis=0)
df_all.drop(['prompt'], axis=1, inplace=True)
df_all

competition label
0                   자꾸 고집부리지마 난 이런 행동 한 적이 없다니까?     1
1                      내가 왜 그런 말을 했다는거야 고집부리지좀 마     1
2              그런 행동이 뭔데? 내가 무슨 잘못을 했다는거지 그만좀 우겨     1
3      너 누가 나한테 그딴 행동이라고 한거야? 그리고 내가 도대체 뭘 믿고 그래     1
4                     내가 너 언제 그랬다고 그래. 오늘이 처음이잖아     1
...                                          ...   ...
11818                         티가 나니까 눈치가 보이는 거죠!     0
11819                              훔쳐보는 거 티나나봐요.     0
11820                                     설렜겠어요.     0
11821                   잘 헤어질 수 있는 사이 여부인 거 같아요.     0
11822                         도피성 결혼은 하지 않길 바라요.     0

[15221 rows x 2 columns]

In [ ]:
tokenized = []
for sent in tqdm(df_all['competition']):
#  tokenized.append(bert_tokenizer(sent))
  t = bert_tokenizer.encode_plus(sent,
                                 add_special_tokens=True,
                                 max_length=30,
                                 pad_to_max_length=True,
                                 return_attention_mask=True)
  tokenized.append(t)

  0%|          | 0/15221 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2393: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 15221/15221 [00:15<00:00, 999.13it/s]


In [ ]:
tokenized[:5]

[{'input_ids': [101, 100, 1455, 30011, 30000, 30019, 30024, 29996, 30014, 29994, 30019, 30000, 30019, 29995, 30006, 1456, 30006, 30021, 1463, 30019, 29994, 30008, 30021, 1469, 30007, 30025, 29993, 30011, 30025, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
 {'input_ids': [101, 1456, 30007, 29991, 30006, 100, 1455, 30017, 29994, 30008, 30021, 1459, 30006, 30022, 29999, 30017, 30022, 100, 1455, 30011, 30000, 30019, 30024, 29996, 30014, 29994, 30019, 30000, 30019, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
 {'input_ids': [101, 1455, 30017, 29994, 30008, 30021, 1469, 30007, 30025, 29993, 30011, 30025, 29999, 30019, 1459, 30015, 300

In [ ]:
input_ids = []
attention_masks = []
for sent in tokenized:
  input_ids.append(sent['input_ids'])
  attention_masks.append(sent['attention_mask'])

input_ids=np.asarray(input_ids)
attention_masks=np.array(attention_masks)

In [ ]:
x_train, x_test, y_train, y_test, mask_train, mask_test = train_test_split(input_ids, df_all['label'], attention_masks, test_size=0.2, random_state=42)
print("x_train: {}개,\ty_train: {}개".format(len(x_train), len(y_train)))
print("x_test: {}개,\ty_test: {}개".format(len(x_test), len(y_test)))
print("mask_train: {}개,\tmask_test: {}개".format(len(mask_train), len(mask_test)))

x_train: 12176개,	y_train: 12176개
x_test: 3045개,	y_test: 3045개
mask_train: 12176개,	mask_test: 3045개


In [ ]:
x_train, x_valid, y_train, y_valid, mask_train, mask_valid = train_test_split(x_train, y_train, mask_train, test_size=0.2, random_state=42)
print("x_train: {}개,\ty_train: {}개".format(len(x_train), len(y_train)))
print("x_valid: {}개,\ty_valid: {}개".format(len(x_valid), len(y_valid)))
print("mask_train: {}개,\tmask_valid: {}개".format(len(mask_train), len(mask_valid)))

x_train: 9740개,	y_train: 9740개
x_valid: 2436개,	y_valid: 2436개
mask_train: 9740개,	mask_valid: 2436개


In [ ]:
bert_model1 = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
bert_model2 = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
bert_model1.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
y_train = pd.DataFrame(y_train)
x_train = pd.DataFrame(x_train)
mask_train = pd.DataFrame(mask_train)

n_train = pd.concat([x_train, mask_train], axis=1)

y_train = y_train['label']
y_t = np.array(y_train)

n_train['label'] = y_t

In [ ]:
nn_train = n_train[n_train['label'] == 1]

In [ ]:
import random
rd_train = nn_train.copy()
rand_num = random.randint(1, 29)

for i in range(len(rd_train)):
    if(int(rd_train.iloc[i, rand_num]) != 0 & int(rd_train.iloc[i, rand_num]) != 102):
      rd_train.iloc[i, rand_num] = 103
    else:
      while(int(rd_train.iloc[i, rand_num]) == 0 & int(rd_train.iloc[i, rand_num]) == 102):
        rand_num = random.randint(1,29)
        if(int(rd_train.iloc[i, rand_num]) != 0 & int(rd_train.iloc[i, rand_num]) != 102):
          rd_train.iloc[i, rand_num] = 103
    rand_num = random.randint(1,29)

In [ ]:
rs_train = nn_train.copy()

rand_1 = random.randint(1, 29)
rand_2 = rand_1
while rand_2 == rand_1:
  rand_2 = random.randint(1, 29)

for i in range(len(rs_train)):
    if (int(rs_train.iloc[i, rand_1]) != 0 & int(rs_train.iloc[i, rand_1]) != 102 & int(rs_train.iloc[i, rand_2]) != 0 & int(rs_train.iloc[i, rand_2]) != 102):
      rs_train.iloc[i, rand_1], rs_train.iloc[i, rand_2] = rs_train.iloc[i, rand_2], rs_train.iloc[i, rand_1]

    else:
      while(int(rd_train.iloc[i, rand_1]) == 0 | int(rd_train.iloc[i, rand_1]) == 102 | int(rs_train.iloc[i, rand_2]) == 0 | int(rs_train.iloc[i, rand_2]) == 102):
        if (int(rs_train.iloc[i, rand_1]) != 0 & int(rs_train.iloc[i, rand_1]) != 102 & int(rs_train.iloc[i, rand_2]) != 0 & int(rs_train.iloc[i, rand_2]) != 102):
          rs_train.iloc[i, rand_1], rs_train.iloc[i, rand_2] = rs_train.iloc[i, rand_2], rs_train.iloc[i, rand_1]
        else:
          rand_1 = random.randint(1, 29)
          rand_2 = rand_1
          while rand_2 == rand_1:
            rand_2 = random.randint(1, 29)

    rand_1 = random.randint(1, 29)
    rand_2 = rand_1
    while rand_2 == rand_1:
      rand_2 = random.randint(1, 29)


1    2167
Name: label, dtype: int64

In [ ]:
values = result_train['label'].value_counts()
values.plot(kind='bar')
print(values)

NameError: ignored

In [ ]:
bert_model2.compile(optimizer = optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = bert_model2.fit([x_train, mask_train],
                         y_train,
                         epochs=5,
                         validation_data=([x_valid, mask_valid], y_valid),
                         batch_size=32)

ValueError: ignored

In [ ]:
loss, acc = bert_sc_model1.evaluate([x_test, mask_test], y_test)

print("loss: ", loss)
print("accuracy: {}%". format(acc))

NameError: ignored